<a href="https://colab.research.google.com/github/JSJeong-me/Machine_Learning/blob/main/ML/10-Bipedalwalker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install system wide packages
Install linux server packages using `apt-get` and Python packages using `pip`

In [1]:
!sudo apt-get update
!apt-get install swig cmake python3-dev libopenmpi-dev zlib1g-dev xvfb x11-utils ffmpeg #remove -qq for full output
!pip install torch stable-baselines3[extra,tests,docs]>=0.10.0 box2d box2d-kengz pyvirtualdisplay pyglet==1.5.0 --quiet #remove --quiet for full output 
%load_ext tensorboard
!rm ./log -r
!rm ./sample_data

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.c

## Import packages
Import python packages

In [2]:
from torch import nn as nn
import gym
import numpy as np
import base64
import IPython
import PIL.Image
import pyvirtualdisplay
import os
# import optuna

from stable_baselines3 import TD3
from stable_baselines3.td3.policies import MlpPolicy
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.env_util import make_vec_env

from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

# Stable baselines 3
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common import results_plotter
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.vec_env import VecVideoRecorder, SubprocVecEnv, DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import CallbackList, BaseCallback, CheckpointCallback, EveryNTimesteps, \
    EvalCallback, StopTrainingOnRewardThreshold, StopTrainingOnMaxEpisodes

# Environment variables

In [3]:
env_id = 'BipedalWalker-v3'
time_steps = 2000000
reward_threshold = 200
episodes_threshold = 1000
video_folder = './videos'
video_length = 3000
logs_base_dir = "./log"
log_dir = "./log"


### Set log dir
os.makedirs(logs_base_dir, exist_ok=True)
### Enviorment 
env = gym.make(env_id)
env = Monitor(env, logs_base_dir)
eval_env = gym.make(env_id)
score = 0
log_interval = 10          # Print avg reward after interval

# TD3 HyperParameters
Here we define our Hyperparamters Twin Delayed DDPG.  
Read more about how you define your TD3 [parameters](https://stable-baselines.readthedocs.io/en/master/modules/td3.html#parameters) 

In [6]:
params = {'batch_size': 256, 'buffer_size': 10000, 'episodic': False, 
          'gamma': 0.99, 'lr': 0.000200462, 'net_arch': 'big', 
          'noise_std': 0.329398, 'train_freq': 128,  'gradient_steps':128, 
          'train_freq':128,'n_episodes_rollout': -1}


net_arch = {
        "small": [64, 64],
        "medium": [256, 256],
        "big":[400, 300] ,
    }[params["net_arch"]]

model = TD3(
        MlpPolicy,
        env,
        gamma=params["gamma"],
        batch_size=params["batch_size"],
        buffer_size=params["buffer_size"],
        train_freq=params["train_freq"],
        learning_rate=params["lr"],
        gradient_steps=params["train_freq"],
        # n_episodes_rollout=params["n_episodes_rollout"],
        policy_kwargs=dict(net_arch=net_arch),
        tensorboard_log=logs_base_dir,
        verbose=0,
    )



## Training & Rec/Play Video [functions]

- `def learning(name, steps=10000, prefix=env_id, eval=1000):`
- `def record(name, length=1500):`  

_that simply help us call the right functions to train our agent and to record & display video_ 

In [7]:
# Training function
def learning(name, steps=10000, prefix=env_id, eval=1000):
  model.learn(total_timesteps=steps, log_interval=log_interval)
  model.save(name + "-" + prefix)
  # Random Agent, after training
  # mean_reward_after_train = evaluate(model, num_steps=eval)


def record(name, length=1500):
   record_video(env_id, model, video_length=length, prefix=name)
   show_videos('videos', prefix=name)
   print(name, " steps total")

## Functions





In [8]:
### Record & Display Video
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

# Record video
def record_video(env_id, model, video_length=500, prefix='', video_folder=video_folder):
  """
  :param env_id: (str)
  :param model: (RL model)
  :param video_length: (int)
  :param prefix: (str)
  :param video_folder: (str)
  """
  env = DummyVecEnv([lambda: gym.make(env_id)])
  # Start the video at step=0 and record 500 steps
  env = VecVideoRecorder(env, video_folder=video_folder,
                              record_video_trigger=lambda step: step == 0, video_length=video_length,
                              name_prefix=prefix)

  obs = env.reset()
  for _ in range(video_length):
    action, _ = model.predict(obs)
    obs, _, _, _ = env.step(action)

  # Close the video recorder
  env.close()


## Display video
def show_videos(video_path='video_folder', prefix=''):
  html = []
  for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
      video_b64 = base64.b64encode(mp4.read_bytes())
      html.append('''<video alt="{}" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>'''.format(mp4, video_b64.decode('ascii')))
  ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

# Display Tensorboard inline
Run & Display tensorboard   
**PS.** *sometimes it does not show up at all, then test to uncomment the reload code, or jusrt run cell again*

It's correctly loaded when you see this view
![Tensorboard](https://nextgrid.ai/wp-content/uploads/2019/12/Screenshot-2019-12-27-at-16.40.02.png)

In [ ]:
# Often not loading on first try, run again until u see the screen
%tensorboard --logdir {logs_base_dir}
# %reload_ext tensorboard


# Custom ballback
Custom callback to messure episodes & score

In [10]:
def evaluate(model, env):
  """
  :param reward_threshold: (int)
  """
  evals = evaluate_policy(model, env, n_eval_episodes=100, deterministic=True, render=False, callback=None, reward_threshold=None, return_episode_rewards=True)
  evals_mean = np.mean(evals[0])
  print("Score over 100 episodes", evals[0])
  print(evals_mean)

  return evals_mean 

In [11]:
class RewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq: (int)
    :param log_dir: (str) Path to the folder where the model will be saved.
    It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: (int)
    """

    def __init__(self, check_freq: int, log_dir: str, verbose=1):
        super(RewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model')
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

            # Retrieve training reward
            x, y = ts2xy(load_results(self.log_dir), 'timesteps')
            if len(x) > 0:
                global episodes
                global mean_reward
                episodes = len(y)
                # print(episodes)
                mean_reward = np.mean(y[-2:])
                mean_reward = round(mean_reward, 0)
                if self.verbose > 0:
                    print(f"Episodes: {episodes}")
                    print(f"Num steps: {self.num_timesteps}")
                    print(f"Mean Episode reward: {mean_reward:.2f} ")
                    print(f"Last Episode reward: {y[-1]:.2f} ")
                    print("=========== NEXTGRID.AI ================")
                # Report intermediate objective value to Optima and Handle pruning
                # trial.report(episodes, self.num_timesteps)
                # if trial.should_prune():
                #     raise optuna.TrialPruned()

                # New best model, you could save the agent here
                if episodes > episodes_threshold:
                    print("Reward threshold achieved")
                    print("Evaluating model....")
                    evals = evaluate(model, eval_env)
                    
                    print(f"Evaluation over 100 Episodes: {evals:.2f} ")
                    if evals >= reward_threshold:
                        model.save("topdog")
                        return False

                # New best model, you could save the agent here
                if mean_reward > reward_threshold:
                    print("Reward threshold achieved")
                    print("Evaluating model....")
                    evals = evaluate(model, eval_env)
                    print("sghould print eval now")
                    print(evals)
                    print(f"Evaluation over 100 Episodes: {evals:.2f} ")
                    if evals >= reward_threshold:
                        print(f"MISSION COMPLETED 🤖")
                        print(f"Score: {evals:.2f} reached at Episode: {episodes} ")
                        model.save("topdog")
                        return False
                    # record(name=steps_total, length=1750)
                    # ep100 = evaluate_policy(model, eval_env, n_eval_episodes=50, deterministic=True, render=False, callback=None, reward_threshold=None, return_episode_rewards=True)
                    # print("Mean Reward 100 Epispodes: ", ep100[0])

                    # print("<======SCORE======>")
                    # print(score)
                    # if score > reward_threshold:

                    # print("Model saved")
                    # return False

        return True

## Train moodel
Add the amount of total moves that will be run before messuring results with `steps_per_round` parameter.  In `limnit` add the score you want model to reach to end training. If not reached it will simply run another round.

In [12]:
# Evaluate before training
evals = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True, render=False, callback=None, reward_threshold=None, return_episode_rewards=True)
print("Score over 100 episodes", evals[0])
# print(np.mean(evals[0])

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Score over 100 episodes [-119.097425614018, -119.5963300478179, -120.11107635544613, -120.40598590532318, -119.60633509000763, -119.54960234020837, -119.75488828169182, -119.37641721987166, -119.2019037399441, -120.35979383205995]


In [ ]:
callback = RewardCallback(check_freq=1000, log_dir=logs_base_dir)
model.learn(total_timesteps=int(time_steps), callback=callback)

Episodes: 12
Num steps: 1000
Mean Episode reward: -109.00 
Last Episode reward: -102.50 
=========== NEXTGRID.AI ================
Episodes: 25
Num steps: 2000
Mean Episode reward: -130.00 
Last Episode reward: -119.39 
=========== NEXTGRID.AI ================
Episodes: 40
Num steps: 3000
Mean Episode reward: -108.00 
Last Episode reward: -108.30 
=========== NEXTGRID.AI ================
Episodes: 46
Num steps: 4000
Mean Episode reward: -118.00 
Last Episode reward: -123.09 
=========== NEXTGRID.AI ================
Episodes: 46
Num steps: 5000
Mean Episode reward: -118.00 
Last Episode reward: -123.09 
=========== NEXTGRID.AI ================
Episodes: 61
Num steps: 6000
Mean Episode reward: -118.00 
Last Episode reward: -120.77 
=========== NEXTGRID.AI ================
Episodes: 78
Num steps: 7000
Mean Episode reward: -106.00 
Last Episode reward: -106.71 
=========== NEXTGRID.AI ================
Episodes: 100
Num steps: 8000
Mean Episode reward: -106.00 
Last Episode reward: -104.77 


# Evaluation
OpenAI scores is generally messured over 100 epochs. Use code belowe to messure your avarage score over 100 rounds

In [ ]:
evals = evaluate_policy(model, env, n_eval_episodes=100, deterministic=True, render=False, callback=None, reward_threshold=None, return_episode_rewards=True)
print("Score over 100 episodes", evals[0])
print(np.mean(evals[0]))

# Record video

In [ ]:
record_video(env_id, model, video_length=4000, prefix="name")
# show_videos('videos', prefix="name")

In [ ]:
del model
env.reset()